In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

# Utility Functions

In [3]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
    #print("Layer: {}".format(layer_obj.name))
    #print("Layer Start Ind: {}".format(layer_obj.istart))
    #print("Layer End Ind: {}".format(layer_obj.iend))
    #print(z_vals)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=sim_proc.X, axis=0)
    y_integral = intg.trapz(x_integral, x=sim_proc.Y, axis=0)
    return y_integral

def integrate_endpoints(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    Nx = arr_slice.shape[1]
    Ny = arr_slice.shape[2]
    Nz = arr_slice.shape[0]
    extended = np.zeros((Nz, Nx+1, Ny+1), dtype=arr_slice.dtype)
    extended[:, 0:Nx, 0:Ny] = arr_slice
    extended[:, 0:Nx, -1] = arr_slice[:, 0:Nx, 0]
    extended[:, -1, 0:Ny] = arr_slice[:, 0, 0:Ny]
    extended[:, -1, -1] = arr_slice[:, 0, 0]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.append(sim_proc.X, sim_proc.X[-1]+sim_proc.dx)
    y_vals = np.append(sim_proc.Y, sim_proc.Y[-1]+sim_proc.dy)
    z_integral = intg.trapz(extended, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.flux_dict
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod, summed_absorbed_power

# NW w/ Shell

## No Endpoints

In [4]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [5]:
print(sim.X[1] - sim.X[0])
print(sim.Y[1] - sim.Y[0])
print(sim.Z[1] - sim.Z[0])

0.00125
0.00125
0.00473035439137


In [6]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.6889811691984553+0j)),
 'Air_bottom': ((8+0j), (-0.6889811691984552+0j)),
 'ITO': ((7.418002620062556-1.803822018736634e-05j),
  (-0.10698378926127916+1.803822018736634e-05j)),
 'ITO_bottom': ((6.94270709124475-0.004415115410177528j),
  (-0.10822149007597238+0.004415115410177528j)),
 'NW_AlShell': ((6.85940044486363+0.0013275440660109605j),
  (-0.024914843694943206-0.0013275440660109605j)),
 'NW_AlShell_bottom': ((2.46691659696217+0.004273111972458624j),
  (-0.08333950032478386-0.004273111972458624j)),
 'Substrate': ((2.3835770966373815+0j), 0j),
 'Substrate_bottom': ((0.35447519644857267+0j), 0j)}

In [12]:
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 5.715139124220379e-05
Total Transmitted Power = 2.9403925017947856e-05
Total Absorbed Power = 0.0005770493660993665
-------------------------
Layer: ITO
Forward Top: (7.418002620062556-1.803822018736634e-05j)
Backward Top: (-0.10698378926127916+1.803822018736634e-05j)
Forward Bottom: (6.94270709124475-0.004415115410177528j)
Backward Bottom: (-0.10822149007597238+0.004415115410177528j)
Power Entering Layer: (7.5262241101385285-0.004433153630364894j)
Power Leaving Layer: (7.049690880506029-0.004433153630364894j)
Absorbed in Layer: (3.952871031050377e-05+0j)
-------------------------
Layer: Air
Forward Top: (8+0j)
Backward Top: (-0.6889811691984553+0j)
Forward Bottom: (8+0j)
Backward Bottom: (-0.6889811691984552+0j)
Power Entering Layer: (8.688981169198454+0j)
Power Leaving Layer: (8.688981169198456+0j)
Absorbed in Layer: (-1.473498395209201e-19+0j)
-------------------------
Layer: Substrate
Forward Top: (2.3835770966373

In [10]:
sim.get_field()

In [14]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

DEBUG:nanowire.optics.postprocess:Logger initialized
DEBUG:nanowire.optics.postprocess:Adding normEsquared to data dict


Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.0005847596667837173


In [15]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Air': (-1.473498395209201e-19+0j), 'Substrate': (0.00016831519024373612+0j), 'NW_AlShell': (0.0003692054655450965+0j), 'ITO': (3.952871031050377e-05+0j)}
{'Air': 0.0, 'Substrate': 0.00016568604984178314, 'NW_AlShell': 0.00038029434390383053, 'ITO': 3.8779273038103651e-05}


In [18]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: (-1.473498395209201e-19+0j)
Integral Method: 0.0
Diff: (-1.473498395209201e-19+0j)
Percent Diff: (-99.99999999999999-0j)
-------------------------
Layer: Substrate
Flux Method: (0.00016831519024373612+0j)
Integral Method: 0.00016568604984178314
Diff: (2.629140401952981e-06+0j)
Percent Diff: (1.5620339424776457+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003692054655450965+0j)
Integral Method: 0.00038029434390383053
Diff: (-1.1088878358734026e-05+0j)
Percent Diff: (3.0034437172706423+0j)
-------------------------
Layer: ITO
Flux Method: (3.952871031050377e-05+0j)
Integral Method: 3.877927303810365e-05
Diff: (7.494372724001219e-07+0j)
Percent Diff: (1.895931505260817+0j)
-------------------------
Total Percent Difference: (1.3361596316273958+0j)


## Include Endpoints

In [19]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

DEBUG:nanowire.optics.postprocess:Logger initialized
DEBUG:nanowire.optics.postprocess:Adding normEsquared to data dict


Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


Integral Method Total Absorption: 0.0005895955496114667


In [23]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'ITO': (3.952871031050377e-05+0j), 'Air': (-1.473498395209201e-19+0j), 'Substrate': (0.00016831519024373612+0j), 'NW_AlShell': (0.0003692054655450965+0j)}
{'Substrate': 0.00016809086399005116, 'Air': 0.0, 'ITO': 3.978572556053836e-05, 'NW_AlShell': 0.00038171896006087717}


In [20]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

-------------------------
Layer: Air
Flux Method: (-1.473498395209201e-19+0j)
Integral Method: 0.0
Diff: (-1.473498395209201e-19+0j)
Percent Diff: (-99.99999999999999-0j)
-------------------------
Layer: Substrate
Flux Method: (0.00016831519024373612+0j)
Integral Method: 0.00016809086399005116
Diff: (2.2432625368495993e-07+0j)
Percent Diff: (0.1332774857457099+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003692054655450965+0j)
Integral Method: 0.00038171896006087717
Diff: (-1.2513494515780667e-05+0j)
Percent Diff: (3.3893037031035527+0j)
-------------------------
Layer: ITO
Flux Method: (3.952871031050377e-05+0j)
Integral Method: 3.978572556053836e-05
Diff: (-2.570152500345872e-07+0j)
Percent Diff: (0.6501989263390963+0j)
-------------------------
Total Percent Difference: (2.1741958745988375+0j)


# NW w/ Shell w/ Lanczos

In [25]:
conf = Config('AbsorptionTest.yml')
conf[('Solver', 'LanczosSmoothing')] = True
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

## No Endpoints

In [26]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.676539839019449+0j)),
 'Air_bottom': ((8+0j), (-0.676539839019449+0j)),
 'ITO': ((7.3740082923072965-0.0007174984378887084j),
  (-0.050548131327010415+0.0007174984378887084j)),
 'ITO_bottom': ((6.8982844387204985-0.0024152328326099703j),
  (-0.05119776341285657+0.0024152328326099703j)),
 'NW_AlShell': ((6.882626480026488+0.00501125688520776j),
  (-0.035539804718956264-0.00501125688520776j)),
 'NW_AlShell_bottom': ((2.6944980277405444+0.007276885080193779j),
  (-0.0989242740708712-0.007276885080193779j)),
 'Substrate': ((2.5955737536697283+0j), 0j),
 'Substrate_bottom': ((0.38818725085573025+0j), 0j)}

In [27]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 5.611937562200762e-05
Total Transmitted Power = 3.220035966251643e-05
Total Absorbed Power = 0.0005752849470749941
-------------------------
Layer: NW_AlShell
Forward Top: (6.882626480026488+0.00501125688520776j)
Backward Top: (-0.035539804718956264-0.00501125688520776j)
Forward Bottom: (2.6944980277405444+0.007276885080193779j)
Backward Bottom: (-0.0989242740708712-0.007276885080193779j)
Power Entering Layer: (6.981550754097359+0.012288141965401539j)
Power Leaving Layer: (2.7300378324595007+0.012288141965401539j)
Absorbed in Layer: (0.0003526654852388597+0j)
-------------------------
Layer: Substrate
Forward Top: (2.5955737536697283+0j)
Backward Top: 0j
Forward Bottom: (0.38818725085573025+0j)
Backward Bottom: 0j
Power Entering Layer: (2.5955737536697283+0j)
Power Leaving Layer: (0.38818725085573025+0j)
Absorbed in Layer: (0.00018310400238057134+0j)
-------------------------
Layer: Air
Forward Top: (8+0j)
Backward To

In [28]:
sim.get_field()

In [29]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air
Layer: Air
Layer Start Ind: 0
Layer End Ind: 106
[ 0.     0.005  0.009  0.014  0.019  0.024  0.028  0.033  0.038  0.043
  0.047  0.052  0.057  0.061  0.066  0.071  0.076  0.08   0.085  0.09
  0.095  0.099  0.104  0.109  0.114  0.118  0.123  0.128  0.132  0.137
  0.142  0.147  0.151  0.156  0.161  0.166  0.17   0.175  0.18   0.184
  0.189  0.194  0.199  0.203  0.208  0.213  0.218  0.222  0.227  0.232
  0.237  0.241  0.246  0.251  0.255  0.26   0.265  0.27   0.274  0.279
  0.284  0.289  0.293  0.298  0.303  0.307  0.312  0.317  0.322  0.326
  0.331  0.336  0.341  0.345  0.35   0.355  0.36   0.364  0.369  0.374
  0.378  0.383  0.388  0.393  0.397  0.402  0.407  0.412  0.416  0.421
  0.426  0.43   0.435  0.44   0.445  0.449  0.454  0.459  0.464  0.468
  0.473  0.478  0.482  0.487  0.492  0.497]


Layer: ITO
Layer: ITO
Layer Start Ind: 106
Layer End Ind: 170
[ 0.501  0.506  0.511  0.516  0.52   0.525  0.53   0.535  0.539  0.544
  0.549  0.553  0.558  0.563  0.568  0.572  0.577  0.582  0.587  0.591
  0.596  0.601  0.605  0.61   0.615  0.62   0.624  0.629  0.634  0.639
  0.643  0.648  0.653  0.658  0.662  0.667  0.672  0.676  0.681  0.686
  0.691  0.695  0.7    0.705  0.71   0.714  0.719  0.724  0.728  0.733
  0.738  0.743  0.747  0.752  0.757  0.762  0.766  0.771  0.776  0.781
  0.785  0.79   0.795  0.799]


Layer: NW_AlShell
Layer: NW_AlShell
Layer Start Ind: 170
Layer End Ind: 438
[ 0.804  0.809  0.814  0.818  0.823  0.828  0.833  0.837  0.842  0.847
  0.851  0.856  0.861  0.866  0.87   0.875  0.88   0.885  0.889  0.894
  0.899  0.903  0.908  0.913  0.918  0.922  0.927  0.932  0.937  0.941
  0.946  0.951  0.956  0.96   0.965  0.97   0.974  0.979  0.984  0.989
  0.993  0.998  1.003  1.008  1.012  1.017  1.022  1.026  1.031  1.036
  1.041  1.045  1.05   1.055  1.06   1.064  1.069  1.074  1.079  1.083
  1.088  1.093  1.097  1.102  1.107  1.112  1.116  1.121  1.126  1.131
  1.135  1.14   1.145  1.149  1.154  1.159  1.164  1.168  1.173  1.178
  1.183  1.187  1.192  1.197  1.202  1.206  1.211  1.216  1.22   1.225
  1.23   1.235  1.239  1.244  1.249  1.254  1.258  1.263  1.268  1.272
  1.277  1.282  1.287  1.291  1.296  1.301  1.306  1.31   1.315  1.32
  1.324  1.329  1.334  1.339  1.343  1.348  1.353  1.358  1.362  1.367
  1.372  1.377  1.381  1.386  1.391  1.395  1.4    1.405  1.41   1.414
  

Layer: Substrate
Layer: Substrate
Layer Start Ind: 438
Layer End Ind: 649
[ 2.072  2.077  2.081  2.086  2.091  2.096  2.1    2.105  2.11   2.114
  2.119  2.124  2.129  2.133  2.138  2.143  2.148  2.152  2.157  2.162
  2.167  2.171  2.176  2.181  2.185  2.19   2.195  2.2    2.204  2.209
  2.214  2.219  2.223  2.228  2.233  2.237  2.242  2.247  2.252  2.256
  2.261  2.266  2.271  2.275  2.28   2.285  2.289  2.294  2.299  2.304
  2.308  2.313  2.318  2.323  2.327  2.332  2.337  2.342  2.346  2.351
  2.356  2.36   2.365  2.37   2.375  2.379  2.384  2.389  2.394  2.398
  2.403  2.408  2.412  2.417  2.422  2.427  2.431  2.436  2.441  2.446
  2.45   2.455  2.46   2.465  2.469  2.474  2.479  2.483  2.488  2.493
  2.498  2.502  2.507  2.512  2.517  2.521  2.526  2.531  2.535  2.54
  2.545  2.55   2.554  2.559  2.564  2.569  2.573  2.578  2.583  2.588
  2.592  2.597  2.602  2.606  2.611  2.616  2.621  2.625  2.63   2.635
  2.64   2.644  2.649  2.654  2.658  2.663  2.668  2.673  2.677  2.682
  2.

In [30]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'ITO': (3.9515459455536426e-05+0j), 'Air': 0j, 'Substrate': (0.00018310400238057134+0j), 'NW_AlShell': (0.0003526654852388597+0j)}
{'Substrate': 0.00018024938435423017, 'Air': 0.0, 'ITO': 3.876789678546686e-05, 'NW_AlShell': 0.00037072187360609352}


In [31]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: ITO
Flux Method: (3.9515459455536426e-05+0j)
Integral Method: 3.876789678546686e-05
Diff: (7.475626700695657e-07+0j)
Percent Diff: (1.8918233024994635+0j)
-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.00018310400238057134+0j)
Integral Method: 0.00018024938435423017
Diff: (2.8546180263411736e-06+0j)
Percent Diff: (1.5590145432255549+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003526654852388597+0j)
Integral Method: 0.0003707218736060935
Diff: (-1.805638836723379e-05+0j)
Percent Diff: (5.11997604614022+0j)


## Include Endpoints

In [32]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


In [33]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'ITO': (3.9515459455536426e-05+0j), 'Air': 0j, 'Substrate': (0.00018310400238057134+0j), 'NW_AlShell': (0.0003526654852388597+0j)}
{'Substrate': 0.00018286540188290699, 'Air': 0.0, 'ITO': 3.9774638267227586e-05, 'NW_AlShell': 0.00037211066850548647}


In [34]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: ITO
Flux Method: (3.9515459455536426e-05+0j)
Integral Method: 3.9774638267227586e-05
Diff: (-2.5917881169116056e-07+0j)
Percent Diff: (0.6558921881771201+0j)
-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: Substrate
Flux Method: (0.00018310400238057134+0j)
Integral Method: 0.000182865401882907
Diff: (2.386004976643501e-07+0j)
Percent Diff: (0.13030872868001675+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003526654852388597+0j)
Integral Method: 0.00037211066850548647
Diff: (-1.9445183266626742e-05+0j)
Percent Diff: (5.513775540993628+0j)


# NW no Shell

In [ ]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [ ]:
sim.get_fluxes()

In [ ]:
abs_dict_fluxmethod = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

In [ ]:
print(sim.data['fluxes'])

In [ ]:
print(sim.flux_dict)

In [ ]:
simulation = Simulation(simulator=sim)
print(simulation.data['fluxes'])
print(simulation.layers)

In [ ]:
alist = [1., 2., 3.]
print('{}, {}, {}, {}'.format('test', *alist))